In [1]:
import json
import sqlite3
import numpy as np
import pandas as pd

try:
  con = sqlite3.connect(database = "data.db")
  cursor = con.cursor()
  print("Successfully connected to database!")

  db_version = "select sqlite_version();"
  cursor.execute(db_version)
  record = cursor.fetchall()
  print(f"SQLite Database Version is: {record[0]} and installed version {sqlite3.sqlite_version}")

except sqlite3.Error as error:
  raise ValueError(f"The following error: {error}, occurred when trying to connect to database!")

Successfully connected to database!
SQLite Database Version is: ('3.40.0',) and installed version 3.40.0


In [2]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

tblLevel
tblLevelList
tblLevelListSelection
tblUser
syncLog
synctblUser
synctblLevel
synctblLevelList
synctblLevelVersion
synctblLevelListSelection
tblLevelVersionScore
tblDevSettings
tblPlayerSettings
tblLevelVersion
tblLevelVersionAutoPlay
synctblLevelVersionAutoPlay
tblEconomy
tblPresentType
tblShopOffer
tblShopOfferBooster
synctblEconomy
synctblPresentType
synctblShopOffer
synctblShopOfferBooster


In [3]:
def preprocessing_grid(data:str,cols:int,rows:int):
    tags = set()
    coords = []

    for j in data.split("},"):
        j += "}"
        j = j.split(":{")[1].split("}")[0]
        tags.add(j.split("tag")[1].split(",")[0][3:-1])
        coords.append(j)

    tags = {x:i for i,x in enumerate(tags)}
    grid = [[0 for x in range(cols)] for y in range(rows)]
    for c in coords:
        row = int(c.split("row")[1].split(":")[1][0])
        col = int(c.split("column")[1].split(":")[1][0])
        tag = c.split("tag")[1].split(":")[1].split(",")[0][1:-1]
        grid[row][col] = tags[tag]
    return np.array([np.array(row) for row in grid])


def preprocessing_mechs(data:str,mechanics,cols:int,rows:int):
    if len(data) < 5:
        return [[0 for x in range(cols)] for y in range(rows)]
    coords = []
    for j in data.split("},"):
        j += "}"
        j = j.split(":{")[1].split("}")[0]
        coords.append(j)

    grid = [[0 for x in range(cols)] for y in range(rows)]
    for c in coords:
        # there is an extra space in some of the coords
        try:
            row = int(c.split("row")[1].split(":")[1][0])
        except:
            row = int(c.split("row")[1:3][1].split(":")[1][0])
        col = int(c.split("column")[1].split(":")[1][0])
        tag = c.split("tag")[1].split(":")[1].split(",")[0][1:-1]
        if tag in mechanics:
            grid[row][col] = mechanics[tag]
        else:
            pass 
    return np.array([np.array(row) for row in grid])

def preprocessing_colors(data:str,cols:int, rows:int):
    if len(data) < 5:
        return [[0 for x in range(cols)] for y in range(rows)]
    coords = []
    for j in data.split("},"):
        j += "}"
        j = j.split(":{")[1].split("}")[0]
        coords.append(j)

    grid = [[0 for x in range(cols)] for y in range(rows)]
    for c in coords:
        # there is an extra space in some of the coords
        try:
            row = int(c.split("row")[1].split(":")[1][0])
        except:
            row = int(c.split("row")[1:3][1].split(":")[1][0])
        col = int(c.split("column")[1].split(":")[1][0])
        tag = c.split("tag")[1].split(":")[1].split(",")[0][1:-1]
        if tag.isdigit():
            grid[row][col] = 1
    return np.array([np.array(row) for row in grid])

In [4]:
q = """ select * from tblLevelVersion"""
df = pd.read_sql(q,con = con)

with open('mechanisms.json','r') as f:
    mechanics = json.load(f)

cols = df["columns"].max(); rows = df["rows"].max()
data = []

exceptions = 0 
for i in range(len(df)):
    try:
        grid = preprocessing_grid(df["boardDefinitionJSON"][i], cols = cols, rows = rows)
        mechs = preprocessing_mechs(df["pieces"][i],mechanics, cols = cols, rows = rows)
        colors = preprocessing_colors(df["pieces"][i], cols = cols, rows = rows)
        board = np.array([grid,mechs,colors])
        data.append(board)
    except Exception as e:
        exceptions += 1 
        # exceptions occurr because the board defintion is empty (10 instances)
print(f"{exceptions} exceptions occurred!")
data = np.array(data)
data.shape

10 exceptions occurred!


(1063, 3, 11, 11)

In [5]:
data[15][2]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])